In [6]:
import re, sqlite3
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    Doc
)

segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)

In [5]:
conn = sqlite3.connect('Harry_Potter80_filled.db')
cur = conn.cursor()

In [13]:
poss = ['NUM', 'X', 'ADJ', 'SYM', 'ADV', 'AUX', 'ADP', 'PROPN', 'NOUN', 'SCONJ', 'CCONJ', 'DET', 'PRON', 'PART', 'VERB', 'INTJ']

In [102]:
request = input('Введите запрос: ')
request_items = re.split(' ', request)
print(request_items)

Введите запрос: "Гарри" Поттер VERB
['"Гарри"', 'Поттер', 'VERB']


## запрос для двух частей:

In [87]:
first_part_2 = '''
    select * FROM
    (select texts.title, b.sentence, t.word as first, 
        (select s.word from tokens s 
        join main on s.token_id = main.token_id
        join sentences p on p.sentence_id = main.sentence_id
        where s.token_id = t.token_id + 1 
        and s'''

second_part_2 = ''' and b.sentence_id = p.sentence_id) second
    from 
    tokens t
    join main on t.token_id = main.token_id
    join sentences b on b.sentence_id = main.sentence_id
    join texts on b.text_id = texts.text_id
    where t'''

third_part_2 = ''')
    where not (second is null)'''

## запрос для трёх частей

In [104]:
first_part_3 = '''
    select * FROM
    (select t.token_id, t.word,
        (select s.word
        from tokens s
        join main on s.token_id = main.token_id
        join sentences p on p.sentence_id = main.sentence_id
        where s.token_id = t.token_id + 1
        and s'''

second_part_3 = ''' and b.sentence_id = p.sentence_id) second,
        (select s.word from tokens s
        join main on s.token_id = main.token_id
        join sentences p on p.sentence_id = main.sentence_id
        where s.token_id = t.token_id + 2
        and s'''
   
third_part_3 = ''' and b.sentence_id = p.sentence_id) third
    from 
    tokens t
    join main on s.token_id = main.token_id
    join sentences p on p.sentence_id = main.sentence_id
    where t'''

fourth_part_3 = ''')
    where not (second is null and third is null)'''

In [91]:
def for_item_maker(item, num):
    for_item = ''
    if '"' in item:
        exact_form = re.search('[а-яА-ЯёЁ]+', item)[0]
        for_item = '.word = ' + exact_form

    elif item in poss:
        for_item = '.pos = ' + item

    elif '+' in item:
        word = re.search('[а-яА-ЯёЁ]+', item)[0]
        pos = re.search('[A-Z]+', item)[0]
        doc = Doc(word)
        doc.segment(segmenter)
        doc.tag_morph(morph_tagger)
        for token in doc.tokens:
            token.lemmatize(morph_vocab)
            lemma = token.lemma
            if num == 0:
                for_item = '.lemma = ' + lemma + ' and t.pos = ' + pos
            else:
                for_item = '.lemma = ' + lemma + ' and s.pos = ' + pos

    else:
        doc = Doc(item)
        doc.segment(segmenter)
        doc.tag_morph(morph_tagger)
        for token in doc.tokens:
            token.lemmatize(morph_vocab)
            lemma = token.lemma
            for_item = '.lemma = ' + lemma
    return for_item

In [106]:
def compilator(request_items=request_items):
    for_items = []
    if len(request_items) == 2:
        for num, item in enumerate(request_items):
            for_item = for_item_maker(item, num)
            for_items.append(for_item)
        query = first_part_2 + for_items[0] + second_part_2 + for_items[1] + third_part_2
    
    elif len(request_items) == 3:
        for num, item in enumerate(request_items):
            for_item = for_item_maker(item, num)
            for_items.append(for_item)
        query = first_part_3 + for_items[0] + second_part_2 + for_items[1] + third_part_3 + for_items[2] + fourth_part_3
    return query

In [107]:
print(compilator())


    select * FROM
    (select t.token_id, t.word,
        (select s.word
        from tokens s
        join main on s.token_id = main.token_id
        join sentences p on p.sentence_id = main.sentence_id
        where s.token_id = t.token_id + 1
        and s.word = Гарри and b.sentence_id = p.sentence_id) second
    from 
    tokens t
    join main on t.token_id = main.token_id
    join sentences b on b.sentence_id = main.sentence_id
    join texts on b.text_id = texts.text_id
    where t.lemma = поттер and b.sentence_id = p.sentence_id) third
    from 
    tokens t
    join main on s.token_id = main.token_id
    join sentences p on p.sentence_id = main.sentence_id
    where t.pos = VERB)
    where not (second is null and third is null)
